In [1]:
#Initially load this data

df_source = spark.createDataFrame([
    (1, 'Laptop', 1200),
    (2, 'Phone', 800),
    (3, 'Tablet', 450)
], ['product_id', 'product_name', 'price'])

StatementMeta(, , -1, SessionStarting, , SessionStarting)

In [ ]:
# Later load this data

df_source = spark.createDataFrame([
    (1, 'Laptop', 1200),
    (2, 'iPhone', 800),
    (4, 'iPad', 650)
], ['product_id', 'product_name', 'price'])

StatementMeta(, a9312347-ed28-4d17-b013-d0fb1e3ff083, 3, Finished, Available, Finished)

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql import SparkSession

target_table = "products_data"
candidate_key = ['product_id']

def loading_pattern_sample(df_source):
    try:
        deltaTable = DeltaTable.forName(spark, target_table)
    except Exception:
        try:
            df_source.write.format("delta").mode("overwrite").saveAsTable(target_table)
            print(f"Created new Delta table '{target_table}' with initial data.")
            return
        except Exception as e:
            print(f"Failed to create table '{target_table}': {str(e)}")
            raise

    try:
        change_detection_columns = [col for col in df_source.columns if col not in candidate_key]
        match_condition = ' AND '.join([f'target.{col} = source.{col}' for col in candidate_key])
        update_condition = ' OR '.join([f'target.{col} != source.{col}' for col in change_detection_columns])
        update_expr = {col: f'source.{col}' for col in df_source.columns}

        merge_operation = deltaTable.alias('target').merge(
            source=df_source.alias('source'),
            condition=match_condition
        ).whenMatchedUpdate(
            condition=update_condition,
            set=update_expr
        ).whenNotMatchedInsertAll()

        merge_operation.execute()
        print(f"Merge into '{target_table}' completed successfully.")
    except Exception as e:
        print(f"Failed to merge data into '{target_table}': {str(e)}")

loading_pattern_sample(df_source)

StatementMeta(, a9312347-ed28-4d17-b013-d0fb1e3ff083, 4, Finished, Available, Finished)

Merge into 'products_data' completed successfully.
